<a href="https://colab.research.google.com/github/ZeleOeO/Artificial-Intelligence-Projects/blob/main/Poem_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import numpy as np

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/irish-lyrics-eof.txt \
    -O /tmp/irish-lyrics-eof.txt

--2023-03-30 18:38:13--  https://storage.googleapis.com/learning-datasets/irish-lyrics-eof.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.201.128, 74.125.202.128, 74.125.69.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.201.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68970 (67K) [text/plain]
Saving to: ‘/tmp/irish-lyrics-eof.txt’

/tmp/irish-lyrics-e 100%[===================>]  67.35K  --.-KB/s    in 0.001s  

2023-03-30 18:38:13 (84.8 MB/s) - ‘/tmp/irish-lyrics-eof.txt’ saved [68970/68970]



In [3]:
tokenizer = Tokenizer()

data = open("/tmp/irish-lyrics-eof.txt").read()
corpus = data.lower().split("\n")

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index)+1


In [4]:
input_sentences = []

#[51, 12, 96, 1217, 48, 2, 69]

for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sentences.append(n_gram_sequence)

In [5]:
max_len = max([len(x) for x in input_sentences])

In [6]:
input_sequences = np.array(pad_sequences(input_sentences, maxlen=max_len, padding="pre"))

In [7]:
xs = input_sequences[:,:-1]
labels = input_sequences[:,-1]


In [8]:
ys = keras.utils.to_categorical(labels, num_classes=total_words)

In [17]:
model = keras.Sequential([
    keras.layers.Embedding(total_words, 240, input_length=max_len-1),
    keras.layers.Bidirectional(keras.layers.LSTM(150)),
    keras.layers.Dense(total_words, activation="softmax")
])

adam = keras.optimizers.legacy.Adam(lr=0.01)
model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=["accuracy"])

model.fit(xs, ys, epochs=700, verbose=1)

Epoch 1/700
377/377 [==============================] - 5s 7ms/step - loss: 6.6450 - accuracy: 0.0779
Epoch 2/700
377/377 [==============================] - 2s 6ms/step - loss: 5.7158 - accuracy: 0.1164
Epoch 3/700
377/377 [==============================] - 3s 7ms/step - loss: 4.8584 - accuracy: 0.1667
Epoch 4/700
377/377 [==============================] - 3s 7ms/step - loss: 4.0180 - accuracy: 0.2304
Epoch 5/700
377/377 [==============================] - 2s 7ms/step - loss: 3.3271 - accuracy: 0.3054
Epoch 6/700
377/377 [==============================] - 2s 6ms/step - loss: 3.2974 - accuracy: 0.3250
Epoch 7/700
377/377 [==============================] - 2s 6ms/step - loss: 2.7474 - accuracy: 0.3975
Epoch 8/700
377/377 [==============================] - 3s 8ms/step - loss: 2.2085 - accuracy: 0.4823
Epoch 9/700
377/377 [==============================] - 3s 7ms/step - loss: 1.9337 - accuracy: 0.5436
Epoch 10/700
377/377 [==============================] - 2s 7ms/step - loss: 1.7276 - accura

In [20]:
seed_text = "I am in love with her"
next_words = 20

for i in range(next_words):
  token_list = tokenizer.texts_to_sequences([seed_text])[0]
  token_list= pad_sequences([token_list], maxlen=max_len-1, padding="pre")
  predicted = np.argmax(model.predict(token_list), axis=-1)
  output_word=""
  for word, index in tokenizer.word_index.items():
    if index==predicted:
      output_word=word
      break
  seed_text += " "+output_word

print(seed_text)

1/1 [==============================] - 0s 19ms/step
I am in love with her my love to find me never you mind when neatest and down again late ones are find by killarneys green
